In [1]:
from nilearn import image as nimg
from nilearn import plotting as nplot
import numpy as np
import pandas as pd
from bids import BIDSLayout

ModuleNotFoundError: No module named 'bids'

In [ ]:
#inputs
fmriprep_dir=''

layout = BIDSLayout(fmriprep_dir,
                    config=['bids', 'derivatives'])

In [ ]:
#load in parcellation
parcel_file = 'tpl-MNI152NLin2009cAsym_res-01_atlas-smorgasbord_dseg.nii.gz'
smorgasbord = nimg.load_img(parcel_file)

mask_labels = open('tpl-MNI152NLin2009cAsym_res-01_atlas-smorgasbord_dseg.tsv')


mask_labels = mask_labels.read().split('\n')
for idx, row in enumerate(mask_labels):
    mask_labels[idx] = mask_labels[idx].split('\t')

In [ ]:
#masker
from nilearn import input_data


masker = input_data.NiftiLabelsMasker(labels_img=smorgasbord,
                                      standardize=True,
                                      memory='nilearn_cache',
                                      verbose=5,
                                      detrend=False,
                                     low_pass = 0.125,
                                     high_pass = 0.042,
                                     t_r=1.0)